In [ ]:
from helpers import *
from pipeline import *
from visualization import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML
import os 
import pipeline as pl
print(os.path.abspath(pl.__file__))

In [ ]:
path = '../../data/080322-ENU-Mouse-Normalisation-Data-Gated.csv'
import pandas as pd
df = pd.read_csv(path, sep=',')
columns_names = df.columns.tolist()
sample_code_column= 'sample_code'
samples_codes = df[sample_code_column]
samples_codes = np.unique(samples_codes)
unique_samples_codes = np.unique([e.split('-')[0]for e in samples_codes])
   
X, Y, columns_names,_ = load_data(path, unique_samples_codes, sample_code_column= 'sample_code')

In [ ]:
pipeline = Pipeline(X, Y, 15, channel_names=columns_names)

In [ ]:
pipeline.segment_data()

In [ ]:
pipeline.commit_changes()

In [ ]:
def interact_pick_channel_segment(pipeline, channel):
    channel = columns_names.index(channel)-1
    area_threshold_widget  = widgets.FloatSlider(min=0.0, max=0.05, step=0.001, value=pipeline.area_thresholds[channel], )
    width_threshold_widget = widgets.FloatSlider(min=0.0, max=0.6, step=.025, value=pipeline.width_thresholds[channel])
    depth_threshold_widget = widgets.FloatSlider(min=0.0, max=0.7, step=.025, value=pipeline.depth_thresholds[channel])

    interact(interactive_update_segments,
         area_threshold = area_threshold_widget,
         width_threshold = width_threshold_widget,
         depth_threshold = depth_threshold_widget,
         channel = fixed(channel),
         pipeline=fixed(pipeline))

def interactive_update_segments(pipeline, channel, area_threshold, width_threshold,
                                depth_threshold):

        pipeline.resegment_channel(channel,area_threshold, width_threshold,depth_threshold)


        plot_channel_segments(ch=channel,
                              samples=pipeline.samples,
                               max_per_plt=20,show_labels=False,
                              limit_x0=-.2,limit_x1=1.2)


In [ ]:
ch_widget = widgets.ToggleButtons(
    options=columns_names[1:-2],
    description='Channel',
    disabled=False,
    style={"button_width": "100px"}
)

ch = ch_widget.index

# kde_window_widget = widgets.FloatSlider(min=0.0, max=0.6, step=.025, value=kde_window[ch])
interact(interact_pick_channel_segment,
         channel = ch_widget,
         pipeline=fixed(pipeline))


In [ ]:
def interactive_recompute_and_update_location_hierarchy_and_refs(pipeline, channel, jaccard_thresholds, mode):
    pipeline.recompute_and_update_location_hierarchy_and_refs([channel], jaccard_thresholds)
    plot_gates_dendrogram(pipeline.agg_models_dict[channel],channel,
                          pipeline.jaccard_thresholds[channel], columns_names[channel+1])
    plt.figure()
    plot_gate_as_tiles(pipeline.data_handler, channel, pipeline.Loc_Ref_Dict_All_Ch)
    if mode==1 or mode ==2:
        plt.figure()
        plot_network_graph(channel, pipeline.incidence_matrices[channel], pipeline.channel_names)
        plt.figure()
        sns.heatmap(pipeline.Sim_Matrix_dict[channel], cmap='Blues')
        if mode ==2:
            plot_channel_segments(ch=channel,
                               samples=pipeline.samples,
                               max_per_plt=20,show_labels=False, color_fill_lg= True,
                               limit_x0=-.2,limit_x1=1.2)
def interact_pick_channel_location_grouping(pipeline, channel, mode):
    channel = columns_names.index(channel)-1

    jaccard_threshold_widget = widgets.FloatSlider(min=0.4, max=0.9,
                                                   step=.01, value=pipeline.jaccard_thresholds[channel])


    interact(interactive_recompute_and_update_location_hierarchy_and_refs,
         jaccard_thresholds = jaccard_threshold_widget,
         channel = fixed(channel),
            mode = fixed(mode),
         pipeline=fixed(pipeline))



In [ ]:
pipeline.commit_changes()
ch_widget = widgets.ToggleButtons(
    options=columns_names[1:-2],
    description='Channel',
    disabled=False,
    style={"button_width": "100px"}
)

mode_widget = widgets.Dropdown(
    options=[0, 1, 2],
    value=0,
    description='Mode:',
    disabled=False,
)


interact(interact_pick_channel_location_grouping,
         channel = ch_widget,
         mode = mode_widget,
         pipeline=fixed(pipeline))


In [ ]:
for s in pipeline.samples:
    print(len(s.gates[7]))

In [ ]:
pipeline.Loc_Ref_Dict_All_Ch[11]

In [ ]:
pipeline.commit_changes()
channels = list(range(15))
pipeline.update_morphology_hierarchy_and_refs(channels,1e-3)


In [ ]:
import time
pipeline.commit_changes()

In [ ]:
time_start = time.time()
print(channels)
pipeline.align_samples(channels, verbose=True, subsample_ratio=.5, n_sample=60)
time_end = time.time()

In [ ]:
print(time_end-time_start)

In [ ]:
plot_scatter_aligned_vs_original([[13,14]], pipeline, columns_names=pipeline.channel_names)

In [ ]:
pipeline.files_names = None

In [ ]:
keys = list(pipeline.aligned_samples.keys())
aligned_samples_numbers, aligned_channels = [k[0] for k in keys], [k[1] for k in keys]
aligned_samples_numbers, aligned_channels = np.sort(np.unique(aligned_samples_numbers)), np.sort(np.unique(aligned_channels))
print(aligned_samples_numbers, aligned_channels)
samp_num_ch_dict = { for s in aligned_samples_numbers}


NameError: name 'keys' is not defined

In [ ]:
clean_and_export(pipeline, 'new_csv.csv', format='fcs')

In [ ]:
All_ch_populations = {}
for ch in channels:
    samples_populations = []
    for s, sample in enumerate(pipeline.samples):
        channel_data = sample.data[:, ch]
        population = np.zeros([channel_data.shape[0],1], dtype=np.int)
        for gate in sample.gates[ch]:
            index = np.logical_and(channel_data>=gate.gate[0], channel_data<=gate.gate[1])
            lg = gate.location_group
            population[index] = lg
        print(np.unique(population), np.count_nonzero(population) == channel_data.shape[0])
        samples_populations+=[population]
    All_ch_populations[ch] = samples_populations

In [ ]:
all_pop_Ys = copy.deepcopy(Y.reshape(-1,1))
for ch in channels:
    pop_Y_ch = np.concatenate(All_ch_populations[ch], axis=0)
    print(pop_Y_ch.shape, all_pop_Ys[0].shape)
    all_pop_Ys = np.concatenate([all_pop_Ys, pop_Y_ch], axis=1)


In [ ]:
population_dict = {8:'B220', 10:'CD44',12:'NK1',13:'CD4', 14:'CD3'}
population_header = ['sample_num', *[f'{c}-'+population_dict[c] for c in channels]]
print(population_header)

In [ ]:
np.savetxt(f'population_label.csv',all_pop_Ys,delimiter=',',header=','.join(population_header),
                   fmt=','.join(['%i']*len(population_header)), comments='')

In [ ]:
def minmaxmean_align(x,y, mi, mx, q_alignment, func0 = None):

    '''

    Creates a warping function that soft aligns target boundaries and mean oto corresponding reference values.

    :param x: :obj:`numpy.array(dtype=float)`, shape = [1, n], A 1-D array containing target data.
    :param y: :obj:`numpy.array(dtype=float)`, shape = [1, n], A 1-D array containing reference data.
    :param mi: :obj:`float`, reference segment left quantile.
    :param mx: :obj:`float`, reference segment right quantile.
    :param q_alignment: :obj:`float`. A quantile value for robust alignment. Instead of matching min and max values of reference, the function matches q_alignment and 1-q_alignment quantiles.
    :param func0: :obj:`function, the transformation that needs to be applied before the minmax_alignment, typically the inverse probability transform.
    :return: :obj:`function`, a warping function that aligns the boundaries of target to reference gates.
    '''

    #  f(x)   = aa x**3 + alpha * x**2 + beta*x + gamma
    #  f(x)   = alpha * x**2 + beta*x + gamma
    # E(f(x)) = alpha * Ex*2 + beta*EY + gamma = EY
    # f(x_mx) = alpha * x_mx*2 + beta*x_mx + gamma = y_mx
    # f(x_mi) = alpha * x_mi*2 + beta*x_mi + gamma = y_mi

    # TODO: ensure monotonicity. Fix bug with quadratic warping.

    if func0 is None:
        func0 = lambda x:x

    z = func0(x)
    ind = ~np.isnan(z)
    x_mean, y_mean =z[ind].mean(), y.mean()

    # TODO: use the precomputed quantiles for target gate
    x_mx, x_mi = np.quantile(z[ind],1-q_alignment), np.quantile(z[ind], q_alignment)
    M = np.array([
        [x_mx**2, x_mx, 1],
        [x_mi**2, x_mi, 1],
        [z[ind].std()**2 + x_mean**2, x_mean, 1]
    ])
    b = np.array([mx, mi, y_mean])
    s = np.linalg.inv(M)@b.T
    alpha, beta, gamma = s[0], s[1], s[2]
    print('alpha',alpha, 'beta',beta, 'gamma',gamma)
    return lambda x: alpha * func0(x)**2 + beta * func0(x) + gamma

def linear_reg_inter(ymean, xmean, x0, y0):

    '''

    :param ymean:
    :param xmean:
    :param x0:
    :param y0:
    :return:
    '''

    a = (ymean - y0)/(xmean-x0)
    f = lambda x: a*x + y0-a*x0
    return f

# def comb_func_factory(x, f_start, f_middle, f_end, x_cut_point_start, x_cut_point_end):
#     ind_st = x<x_cut_point_start
#     ind_end = x>x_cut_point_end
#     ys = f_start(x[ind_st])
#     ym = f_middle(x[~np.logical_or(ind_st, ind_end)])
#     ye = f_end(x[ind_end])
#     return np.concatenate([ys, ym, ye])
#
#
# def combine_funcs(funcs, locations, smooth_sigma=0.1):
#     xcom = []
#     ycom = []
#     plot = True
#     for i, f in enumerate(funcs):
#         g0, g1 = locations[i]
#         print(g0, g1)
#         x_s = np.linspace(g0, g1, 2000)
#         y_s = f(x_s)
#         xcom += [x_s]
#         ycom += [y_s]
#         if plot:
#             plt.plot(x_s, y_s)
#             plt.xlim(-.3,1.3)
#             plt.ylim(-.3,1.3)
#     xcom = np.concatenate(xcom)
#     ycom = np.concatenate(ycom)
#     # middle
#     f_middle = scipy.interpolate.interp1d(xcom, ycom, kind='cubic')
#
#     # start
#     x_cut_point_end = locations[0][0] + .1*np.abs(locations[0][0]-locations[0][1])
#     ind = np.argmin(np.abs(xcom-x_cut_point_end))
#     x_cut_point_start = xcom[ind]
#     y_cut_point_start = ycom[ind]
#     ind = xcom<=x_cut_point_start
#     x, y = xcom[ind], ycom[ind]
#     f_start = linear_reg_inter(y.mean(), x.mean(), x_cut_point_start, y_cut_point_start)
#     x = np.linspace(-.3, x_cut_point_start, 20000)
#     if plot:
#         plt.figure()
#         plt.plot(x, f_start(x))
#
#     # end
#     x_cut_point_end = locations[-1][1] - .1*np.abs(locations[-1][0]-locations[-1][1])
#     ind = np.argmin(np.abs(xcom-x_cut_point_end))
#     x_cut_point_end = xcom[ind]
#     y_cut_point_end = ycom[ind]
#     ind = xcom >= x_cut_point_end
#     x, y = xcom[ind], ycom[ind]
#     f_end = linear_reg_inter(y.mean(), x.mean(), x_cut_point_end, y_cut_point_end)
#     if plot:
#         x = np.linspace(x_cut_point_end, 1.3, 20000)
#         plt.plot(x, f_end(x))
#         x = np.linspace(x_cut_point_start, x_cut_point_end, 20000)
#         plt.plot(x, f_middle(x))
#
#     comb_func = lambda x: gaussian_filter1d(comb_func_factory(x, f_start, f_middle, f_end,
#                                                               x_cut_point_start, x_cut_point_end),smooth_sigma)
#     return comb_func
#

def lambda_IPT_factory(ref_inv_cdf, target_ecdf):

    '''
    A factory method for Inverse Probability Transform. The reason why it is necessary to have this kind of factory/additional layer is that
    python lambdas do not store default hyperparamters for non-primitive data types and will access the most recent parameters in its closure.
    By having a factory, we ensure that parameters are snapshotted in the scope of the factory and the returned lambda closure has its own unique hyperparameters.

    :param ref_inv_cdf: :obj:`function`, inverse empirical cumulative distribution function (ECDF) of reference segment
    :param target_ecdf: :obj:`function`, ECDF of target segment.
    :return: :obj:`function`, inverse probability transform.
    '''

    return lambda x: ref_inv_cdf(target_ecdf(x))

def compute_cdf_and_inv_cdf(ref_gate, k=3):

    '''

    Computes the CDF and Inverse CDF of a gate. (typically for a reference gate)

    :param ref_gate: :obj:`Gate`, reference gate.
    :param k: :obj:`int`, BSpline degree (default =3).
    :returns:
         - ref_ecdf - :obj:`function`, reference Empirical Cumulative distribution function.
         - ref_inv_cdf - :obj:`function`, reference inverse Cumulative distribution function.

    '''

    ref_seg = ref_gate.segment
    # Compute empirical cumulative distribution function for the reference segment
    ref_ecdf = sm.distributions.empirical_distribution.ECDF(ref_seg)
    # Compute the inverse of the ECDF of reference using monotone function inverter
    ref_inv_cdf = sm.distributions.empirical_distribution.monotone_fn_inverter(ref_ecdf,
                                                                               ref_seg)
    # fit a BSpline tp inverse ECDF

    ref_inv_cdf = scipy.interpolate.BSpline(
        ref_inv_cdf.x,
        ref_inv_cdf.y,
        k=k,
        extrapolate=False
    )
    return ref_ecdf, ref_inv_cdf

def get_smooth_and_monotonic(t, y):

    '''

    A convolution-based transform to ensure smoothness and monotonicity.

    :param t:
    :param y:
    :return:
    '''

    N = y.shape[0]//10
    yy =  np.convolve(y, np.ones(N)/N, mode='valid')
    yy, idx = np.unique(yy, return_index=True)
    tt = t[idx]
    return np.sort(tt), np.sort(yy), [tt.min(), tt.max()]

def combined(x, model, funcs, locations, sigma, only_use_MARS = False ):

    '''

    The final combined function of MARS model and segments transformations.

    :param x: :obj:`numpy.array(dtype=float)`, shape = [1, n], A 1-D array containing target data.
    :param model: model: :obj:`pyearth.Earth` MARS model.
    :param funcs: obj:`list(function)`, a list of warping functions for each segment.
    :param locations: obj:`list(tuple)`, a list of tuples that contain the start and end range of each function.
    :return: y: :obj:`numpy.array(dtype=float)`, shape = [1, n], A 1-D array containing transformed target data.
    '''

    if only_use_MARS:
        y = model.predict(x)
        return y

    y = np.zeros(x.shape)
    all_inds = []
    for i, g in enumerate(locations):
        ind = np.logical_and(x>=g[0], x<g[1])
        f = funcs[i]
        if f is not None:
            y[ind] =  gaussian_filter1d( f(x[ind]), sigma )
            all_inds+=[ind]
    if len(all_inds)>1:
        remaining_ind = ~np.logical_or(*all_inds)
    else:
        remaining_ind = ~all_inds[0]

    if np.count_nonzero(remaining_ind)>0:
        y[remaining_ind] = model.predict(x[remaining_ind])
    return y

def combine_funcs(model, xs_ys, funcs, locations, sigma, subsample_ratio=1, plot = True, ax = None):

    '''

    Creates a combined function of MARS model and every segments transformations.

    :param model: :obj:`pyearth.Earth` MARS model.
    :param xs_ys: obj:`list(list(numpy.array))`, a list of x values and transformed y values in the form of [ [target_seg1, f(target_seg1)], [target_seg2, f(target_seg2)], ...].
    :param funcs: obj:`list(function)`, a list of warping functions for each segment.
    :param locations: obj:`list(tuple)`, a list of tuples that contain the start and end range of each function.
    :param sigma: :obj:`float`, the standard deviation of Gaussian kernel.
    :param subsample_ratio: :obj:`float`, a number between [0,1]`, represents the ratio of cells to consider to estimate MARS parameters.
    :param plot: :obj:`Bool`, if :obj:`True`, plots warping functions.
    :param ax: :obj:`matplotlib.axes.Axes`, axes to draw on. If :obj:`None`, a new axes is created.
    :return: :obj:`function`, the complete warping function for a sample's channel including its MARS regression model for extrapolation.
    '''
    xcom = []
    ycom = []

    if plot and ax is None:
        ax = plt.subplots(1,1)[1]
    for i, xy in enumerate(xs_ys):
        x_s, y_s,= xs_ys[i][0],xs_ys[i][1]
        if 0<subsample_ratio<1:
            m = x_s.shape[0]
            indx = np.random.randint(0, m, int(m*subsample_ratio))
            xcom += [x_s[indx]]
            ycom += [y_s[indx]]
        elif subsample_ratio == 1:
            xcom += [x_s]
            ycom += [y_s]
        else:
            raise Exception(f'subsample_ratio must be >0 and <=1, you passed {subsample_ratio}')
        if plot:
            ax.plot(x_s, y_s, linewidth=3)
    xcom = np.concatenate(xcom)
    ycom = np.concatenate(ycom)
    # middle
    model.fit(xcom, ycom)
    comb_func = lambda x: combined(x, model, funcs, locations, sigma)
    if plot:
            t = np.linspace(-.1,1.1,5000)
            ax.plot(t, comb_func(t))
            ax.set_xlim(-.3,1.3)
            ax.set_ylim(-.3,1.3)
    return comb_func



def estimate_alignment_funcs_for_target(ch, target_sample, q_alignment, Loc_Ref_Dict_All_Ch, Ref_Dict_indx_by_Loc_and_Morph, Ref_CDF_and_InvCDF, verbose):


    funcs = []
    z = target_sample(ch)
    sample_orignal = copy.deepcopy(z)
    gates_locations = []
    xys = []

    for i in range(len(target_sample.gates[ch])):
        target_gate = target_sample.gates[ch][i]
        ind = np.logical_and(z >= target_gate.gate[0], z < target_gate.gate[1])
        target_seg1 = target_gate.segment
        lo_gr, mr_gr = target_gate.location_group, target_gate.morphology_group

        ref_gate = Loc_Ref_Dict_All_Ch[ch][lo_gr]
        mr_gr_ref = Ref_Dict_indx_by_Loc_and_Morph.get((lo_gr, mr_gr))


        if verbose:
            print('diff:', np.abs(target_gate.segment - target_seg1).sum())
            print('lo_gr, mr_gr', lo_gr, mr_gr)
            print('\n mr_gr_ref', mr_gr_ref, '\n')
            print('target gate', target_gate, '\n')

        if target_gate.leave_to_MARS:
            funcf = None

        else:
            if mr_gr_ref:
                ref_cdf, ref_inv_cdf = Ref_CDF_and_InvCDF.get((lo_gr, mr_gr))

                if mr_gr_ref != target_gate:
                    # Compute empirical cumulative distribution function for the target segment
                    target_ecdf = sm.distributions.empirical_distribution.ECDF(target_seg1)
                    func0 = lambda_IPT_factory(ref_inv_cdf, target_ecdf)

                else:
                    func0 = lambda x: x

            q0, q1 = ref_gate.get_tight_gates(q=q_alignment)
            if mr_gr_ref:
                funcf = minmaxmean_align(target_seg1, ref_gate.segment, q0, q1, q_alignment, func0)

            else:
                funcf = minmaxmean_align(target_seg1, ref_gate.segment, q0, q1, q_alignment)

        funcs += [funcf]
        if verbose:
            print('loc. gate is alredy ref?', ref_gate == target_gate)
            print('gate:\n', target_gate.gate)
        sample_orignal[ind] = z[ind]
        if not target_gate.leave_to_MARS:
            t = np.linspace(target_gate.tight_gate[0], target_gate.tight_gate[1], 2000)
            y = funcf(t)
            ind = ~np.isnan(y)
            xys += [[t[ind], y[ind]]]
        gates_locations += [[target_gate.gate[0], target_gate.gate[1]]]

    return funcs, xys, gates_locations, sample_orignal


def align_samples_func(ch, q_alignment,
                  samples,
                  aligned_samples,
                  original_samples,
                  Loc_Ref_Dict_All_Ch,
                  Loc_Morph_Ref_Dict_All_Ch,
                  Ref_Inv_CDF_Dict_All_Ch,
                  funcs_dict, comp_func_dict,earth_models_dict,
                  gates_locations_dict,
                  n_sample,
                  sigma, earth_smoothing_penalty, subsample_ratio=1,
                  in_place_eval = True,
                  verbose=False):

    Ref_Dict_indx_by_Loc_and_Morph = Loc_Morph_Ref_Dict_All_Ch[ch]
    Ref_CDF_and_InvCDF = {}
    for (lo_gr, mr_gr), ref_gate in Ref_Dict_indx_by_Loc_and_Morph.items():
        ref_ecdf, ref_inv_cdf = compute_cdf_and_inv_cdf(ref_gate)
        Ref_CDF_and_InvCDF[lo_gr, mr_gr] = (ref_ecdf, ref_inv_cdf)

    Ref_Inv_CDF_Dict_All_Ch[ch] = Ref_CDF_and_InvCDF

    for targ_num in range(n_sample):
        if verbose:
            print(f'Sample {targ_num} **** \n')

        target_sample = samples[targ_num]
        funcs, xys, gates_locations, sample_orignal = estimate_alignment_funcs_for_target(ch,target_sample= target_sample, q_alignment=q_alignment,
                                                                       Loc_Ref_Dict_All_Ch= Loc_Ref_Dict_All_Ch,
                                                                       Ref_Dict_indx_by_Loc_and_Morph=Ref_Dict_indx_by_Loc_and_Morph,
                                                                       Ref_CDF_and_InvCDF=Ref_CDF_and_InvCDF, verbose= verbose )

        model = Earth(penalty=earth_smoothing_penalty, smooth=True, max_degree=1, )
        comp_func = combine_funcs(model, xys, funcs, gates_locations, sigma=sigma, subsample_ratio=subsample_ratio, plot=False)
        comp_func_dict[ch, targ_num] = comp_func
        earth_models_dict[ch, targ_num] = model
        funcs_dict[targ_num, ch] = funcs

        if in_place_eval:
            gates_locations_dict[targ_num, ch] = gates_locations
            sample_aligned = comp_func(sample_orignal)
            aligned_samples[targ_num, ch] = sample_aligned
            original_samples[targ_num, ch] = sample_orignal

    return comp_func_dict, funcs_dict, earth_models_dict

In [ ]:
pipeline.samples[57].gates[10][1].leave_to_MARS

In [ ]:
for ch in range(7,10):
    align_samples_func(ch=ch, q_alignment = pipeline.gate_factor_q[ch],
                       samples=pipeline.samples, aligned_samples=pipeline.aligned_samples,
                       original_samples=pipeline.original_samples,
                       Loc_Ref_Dict_All_Ch=pipeline.Loc_Ref_Dict_All_Ch,Ref_Inv_CDF_Dict_All_Ch=pipeline.Ref_Inv_CDF_Dict_All_Ch,
                       Loc_Morph_Ref_Dict_All_Ch=pipeline.Loc_Morph_Ref_Dict_All_Ch,
                       funcs_dict=pipeline.funcs_dict, comp_func_dict=pipeline.comp_func_dict,
                       gates_locations_dict=pipeline.gates_locations_dict,
                       earth_models_dict=pipeline.earth_models_dict,
                       n_sample = 60, sigma=1,
                       earth_smoothing_penalty = .1,
                       subsample_ratio = .5,
                       verbose = True)

In [ ]:
plot_original_vs_aligned([10],pipeline.samples, pipeline.aligned_samples,
                             pipeline.Loc_Ref_Dict_All_Ch, columns_names,n_sample = -1,
                             max_per_plt=15, root_path = None, xlim0=0, xlim1 = 1,)

In [ ]:
plot_scatter_aligned_vs_original([[10,14]], pipeline, columns_names=pipeline.channel_names)

In [ ]:
aligned_samples_data = []
for s in range(pipeline.num_samples):
    sample = []
    orig_sample = []
    for ch in range(pipeline.num_channels):
        samp_ch = pipeline.aligned_samples[s, ch].reshape(-1, 1)
        sample +=[samp_ch]
    sample = np.concatenate(sample, axis=1 )
    y = np.zeros([sample.shape[0],1], dtype=np.int)+s
    sample = np.concatenate([y, sample], axis = 1)
    aligned_samples_data += [sample]

aligned_samples_data = np.concatenate(aligned_samples_data)
header = ','.join(pipeline.channel_names)
np.savetxt('aligned_data_all_markers.csv',aligned_samples_data,delimiter=',',header=header,
           fmt=','.join(['%i'] + ['%1.4f']*pipeline.num_channels), comments='')


In [ ]:
aligned_samples_data.shape


In [ ]:
print(X.shape)
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# for i in range(60):
for i in [56,57,58]:
    t = np.linspace(0,1.2,2000)
    f = pipeline.comp_func_dict[10, i]
    y = f(t)
    plt.plot(t,y)

In [ ]:
print()
i = 18
plt.scatter(pipeline.aligned_samples[i,0], pipeline.aligned_samples[i,10])

In [ ]:
# data_hist = np.ones([60, 15, 256],dtype=np.int)
data_hist = {}

In [ ]:
for y in range(60):
    sample_hists = np.ones([15, 256], dtype=np.int)
    for i in range(15):
        x = np.histogram(X[Y==y, i], bins=256)[0]
        sample_hists[i, :] = x
    # data_hist[y, :, :] = sample_hists
    data_hist[y] = sample_hists

In [ ]:
print(data_hist.keys())

In [ ]:
import scipy.io as sio

In [ ]:
save_path = '..\\..\\data\\60_mice_15_channels_hists.mat'
np.savetxt(path,data_hist,delimiter=',',header=columns_names[1:-2],
                   fmt=','.join(['%i'] * 15), comments='')

In [ ]:
df = pd.DataFrame(data_hist.items())

In [ ]:
save_path = '..\\..\\data\\60_mice_15_channels_hists.xlsx'
df.to_excel(save_path)

In [ ]:
df1 = pd.read_excel(save_path)

In [ ]:
print(df1[1][1])